### Generate ONNX representations

In [1]:
import torch
import torch.nn.functional as F
import torchvision.models as models
import pandas as pd
import os
import sys
#from utils import *
import json
from PIL import Image
from torchvision import transforms
from scorecamMOD import ScoreCAM, ScoreCAMOnly
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(project_root)
from training.defineSNN2 import SimpleConvNet, BiggerConvNet
#import subprocess
#subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define image preprocessing
preprocess = transforms.Compose([
    transforms.Resize((50, 50)),
    transforms.ToTensor(),
])

img1_path = "TestInvalid_t0.jpeg"
img2_path = "TestInvalid_t1.jpeg"
img1 = Image.open(img1_path).convert('L')
img2 = Image.open(img2_path).convert('L')
img1 = preprocess(img1).unsqueeze(0).to(device)
img2 = preprocess(img2).unsqueeze(0).to(device)
print(img1.size())

torch.Size([1, 1, 50, 50])


# DNN conversion

In [2]:
model_path = os.path.join('outputs/dnn/model23k.onnx')
data_path = os.path.join('outputs/dnn/input23k.json')

loaded_model = BiggerConvNet().to(device)
#loaded_model.load_state_dict(torch.load('training/trained_NEWsnn_lr_0.1B.pth', map_location=torch.device('cpu')))
#loaded_model.load_state_dict(torch.load('training/trained_simplesnn_lr_0.01BIG.pth', map_location=torch.device('cpu')))
#loaded_model.load_state_dict(torch.load('training/trained_3conv_lr_0.1B.pth', map_location=torch.device('cpu')))
#loaded_model.load_state_dict(torch.load('training/trained_SQUEEZEsnn_lr_0.1B.pth', map_location=torch.device('cpu')))
#loaded_model.load_state_dict(torch.load('../training/trained__460__lr_0.001B.pth', map_location=torch.device('cpu')))
loaded_model.load_state_dict(torch.load('../training/trained__23k__lr_0.001B.pth', map_location=torch.device('cpu')))

loaded_model.eval()

# Define image preprocessing
preprocess = transforms.Compose([
    transforms.Resize((50, 50)),
    transforms.ToTensor(),
])

img1_path = "TestInvalid_t0.jpeg"
img2_path = "TestInvalid_t1.jpeg"
img1 = Image.open(img1_path).convert('L')
img2 = Image.open(img2_path).convert('L')
img1 = preprocess(img1).unsqueeze(0).to(device)
img2 = preprocess(img2).unsqueeze(0).to(device)
print(img1.size())

# Export the loaded Siamese Network model to ONNX
torch.onnx.export(loaded_model,                      # loaded Siamese Network model
                  (img1, img2),                      # model input (or a tuple for multiple inputs)
                  model_path,                        # where to save the ONNX model
                  export_params=True,                # store the trained parameter weights inside the model file
                  opset_version=14,                  # the ONNX version to export the model to
                  do_constant_folding=True,          # whether to execute constant folding for optimization
                  input_names = ['input1', 'input2'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input1' : {0 : 'batch_size'}, 'input2' : {0 : 'batch_size'},   # variable length axes
                                'output' : {0 : 'batch_size'}})


# Move tensors from GPU to CPU
img1_cpu = img1.cpu()
img2_cpu = img2.cpu()

# Serialize data into a JSON file
input_data = [
    img1_cpu.detach().numpy().reshape([-1]).tolist(),
    img2_cpu.detach().numpy().reshape([-1]).tolist(),
]

data = dict(input_data=input_data)
json.dump(data, open(data_path, 'w'))

torch.Size([1, 1, 50, 50])


In [3]:
# *** Providing the calibrate settings function with a larger and broader range of sample inputs.

cal_path = os.path.join('outputs/dnn/calib_data23k.json')

path_to_dataset = '../training/dataset1k224x224'

csv_path = os.path.join(path_to_dataset, 'forest_dataset.csv')
df = pd.read_csv(csv_path)

# Construct the full paths to the images
img1_files = [os.path.join('../training/dataset1k224x224', path) for path in df['imageT0'].iloc[:20]]
img2_files = [os.path.join('../training/dataset1k224x224', path) for path in df['imageT1'].iloc[:20]]

img1_list = []
img2_list = []
for img1_path, img2_path in zip(img1_files, img2_files):
    
    img1 = Image.open(img1_path).convert('L')
    img2 = Image.open(img2_path).convert('L')
    img1 = preprocess(img1).unsqueeze(0).to(device)
    img2 = preprocess(img2).unsqueeze(0).to(device)
   #img1, img2 = siamese_loader.load_and_transform_pair(img1_path, img2_path)
    img1 = img1.unsqueeze(0).cpu().detach().numpy().reshape([-1]).tolist()
    img2 = img2.unsqueeze(0).cpu().detach().numpy().reshape([-1]).tolist()
    img1_list.extend(img1) # don't use .append bc it should look like [[xxx...],[yyy...]]
    img2_list.extend(img2)

data = dict(input_data = [img1_list, img2_list])
# Serialize data into file:
json.dump( data, open(cal_path, 'w' ))

# GetScore (for aggregation circuit) conversion

In [ ]:
from prover.scorecamMOD import GetActivations,  GetMask, GetScore, GetCAM
import onnx

loaded_model = SimpleConvNet().to(device)
loaded_model.load_state_dict(torch.load('../training/trained__460__lr_0.001B.pth', map_location=torch.device('cpu')))

# Define image preprocessing
preprocess = transforms.Compose([
    transforms.Resize((50, 50)),
    transforms.ToTensor(),
])

img1_path = "TestInvalid_t0.jpeg"
img2_path = "TestInvalid_t1.jpeg"
img1 = Image.open(img1_path).convert('L')
img2 = Image.open(img2_path).convert('L')
img1 = preprocess(img1).unsqueeze(0).to(device)
img2 = preprocess(img2).unsqueeze(0).to(device)

#######################
loaded_model.eval()

#target_layer = loaded_model.embedding_net.features[4] 
target_layer = loaded_model.conv1

input_size = (50, 50)  # Input image size

activations_extractor = GetActivations(loaded_model, target_layer)
activations = activations_extractor(img1, img2)

with torch.no_grad():
    score = loaded_model(img1, img2)
    print("normal score: ", score)

# resize activation maps
def resize_activations(activations, size):
    resized_activations = []
    for i in range(activations.size(1)):
        resized_activation = transforms.functional.resize(activations[:, i, :, :].unsqueeze(1), size).squeeze(1)
        resized_activations.append(resized_activation)
    return torch.stack(resized_activations, dim=1)
activations = resize_activations(activations, input_size)

masks_generator = GetMask(activations)
scores_generator = GetScore(loaded_model)
scores_generator.eval()


# Export the scores_generator model to ONNX
model_path = os.path.join('outputs/cam/aggr/modelGetscore460.onnx')
torch.onnx.export(scores_generator, (img1, img2), model_path,
                  export_params=True, opset_version=14,
                  do_constant_folding=True, input_names=['masked_input', 'input2'],
                  output_names=['score'], dynamic_axes={'masked_input': {0: 'batch_size'},
                                                        'input2': {0: 'batch_size'},
                                                        'score': {0: 'batch_size'}})

# Create directory for serialized input data if it doesn't exist
input_data_dir = os.path.join('outputs/cam/aggr/inputs')
os.makedirs(input_data_dir, exist_ok=True)


all_scores = []
for index in range(activations.size(1)):  # Loop over the number of channels
    masked_input = masks_generator(img1, index)
    with torch.no_grad():
        score = scores_generator(masked_input, img2) # score_generator OR LOADED MODEL???
        print("score2: ", score)
    all_scores.append(score)
 
    masked_input_cpu = masked_input.cpu()
    img2_cpu = img2.cpu()
    # INPUT SERIALIZATION
    input_data = [
        masked_input_cpu.detach().numpy().reshape([-1]).tolist(),
        img2_cpu.detach().numpy().reshape([-1]).tolist(),
    ]
    
    data = dict(input_data=input_data)
    data_path = os.path.join(input_data_dir, f'input460_{index}.json')
    with open(data_path, 'w') as f:
        json.dump(data, f)

# inspect the onnx model
onnx_model = onnx.load(model_path)
onnx.checker.check_model(onnx_model)
print("Model loaded and checked successfully!")
print(onnx.helper.printable_graph(onnx_model.graph))

In [11]:
# *** Providing the calibrate settings function with a larger and broader range of sample inputs.
cal_path = os.path.join('outputs/cam/aggr/calib_dataGetscore460.json')
path_to_dataset = '../training/dataset1k224x224'
csv_path = os.path.join(path_to_dataset, 'forest_dataset.csv')
df = pd.read_csv(csv_path)


# Construct the full paths to the images
img1_files = [os.path.join('../training/dataset1k224x224', path) for path in df['imageT0'].iloc[:20]]
img2_files = [os.path.join('../training/dataset1k224x224', path) for path in df['imageT1'].iloc[:20]]

img1_list = []
img2_list = []
for img1_path, img2_path in zip(img1_files, img2_files):
    
    img2 = Image.open(img2_path).convert('L')
    img2 = preprocess(img2).unsqueeze(0).to(device)
    size= (img2.size())
    img2 = img2.unsqueeze(0).cpu().detach().numpy().reshape([-1]).tolist()
    
    img1 = torch.rand(size).to(device)  # Random tensor for img1
    img1 = img1.cpu().detach().numpy().reshape([-1]).tolist()
    
    img1_list.extend(img1) # don't use .append bc it should look like [[xxx...],[yyy...]]
    img2_list.extend(img2)

data = dict(input_data = [img1_list, img2_list])
# Serialize data into file:
json.dump( data, open(cal_path, 'w' ))

# GetCam conversion

In [12]:
 # EXPORT the other class GETCAM! ***
cam_generator = GetCAM(activations.size(1))
stacked_scores = torch.stack(all_scores, dim=1)  # Concatenate along the correct dimension

print(stacked_scores.squeeze(2).size())
print(activations.size())
cam = cam_generator(stacked_scores, activations)    

input_data_dir = os.path.join('outputs/cam')
model_path = os.path.join('outputs/cam/modelGetcam460.onnx')
torch.onnx.export(cam_generator, (stacked_scores.squeeze(2), activations), model_path,
                  export_params=True, opset_version=14,
                  do_constant_folding=True, input_names=['all_scores', 'activations'],
                  output_names=['cam'], dynamic_axes={'all_scores': {0: 'batch_size'},
                                                      'activations': {0: 'batch_size'},
                                                      'cam': {0: 'batch_size'}})

# Serialize inputs for ONNX model
all_scores_cpu = stacked_scores.cpu()
activations_cpu = activations.cpu()

# Serialize input data for ONNX
input_data = [
    all_scores_cpu.detach().numpy().flatten().tolist(),
    activations_cpu.detach().numpy().flatten().tolist(),
]

print(len(input_data[0]))
print(len(input_data[1]))


data_path = os.path.join(input_data_dir, 'inputGetcam460.json')
data = dict(input_data=input_data)
json.dump(data, open(data_path, 'w'))
print(f"Serialized inputs for GetCAM exported to {data_path}")

torch.Size([1, 3])
torch.Size([1, 3, 50, 50])
3
7500
Serialized inputs for GetCAM exported to outputs/cam/inputGetcam460.json


In [14]:
# create dummy activations and outputs for Calibration

cal_path = os.path.join('outputs/cam/calib_dataGetcam460.json')
def create_dummy_data(num_samples, scores_shape, activation_shape):
    dummy_activations_list = []
    dummy_scores_list = []

    for _ in range(num_samples):
        # Generate random scores and activations
        dummy_scores = torch.randn(*scores_shape).cpu().detach().numpy().tolist()
        dummy_activations = torch.randn(*activation_shape).cpu().detach().numpy()

        # Flatten nested lists and append scores
        flattened_scores = [item for sublist in dummy_scores for inner_list in sublist for item in inner_list]
        dummy_scores_list.extend(flattened_scores)

        print("dummy_scores_list length: ", len(dummy_scores_list))

        # Flatten activations and append
        dummy_activations_list.extend(dummy_activations.flatten().tolist())

    return dummy_scores_list, dummy_activations_list

# Get the shapes of the inputs
print(all_scores)
stacked_scores = torch.stack(all_scores, dim=1)  
scores_shape = stacked_scores.size()
print(scores_shape)

activation_shape = activations.size()


# Create dummy data
dummy_scores_list, dummy_activations_list = create_dummy_data(20, scores_shape, activation_shape)
print(len(dummy_scores_list))
print(len(dummy_activations_list))

# Save the data to a JSON file
data = {'input_data': [dummy_scores_list, dummy_activations_list]}


json.dump(data, open(cal_path, 'w'))
print(f"Serialized calibration data to {cal_path}")



[tensor([[8.8819]]), tensor([[5.9736]]), tensor([[2.1965]])]
torch.Size([1, 3, 1])
dummy_scores_list length:  3
dummy_scores_list length:  6
dummy_scores_list length:  9
dummy_scores_list length:  12
dummy_scores_list length:  15
dummy_scores_list length:  18
dummy_scores_list length:  21
dummy_scores_list length:  24
dummy_scores_list length:  27
dummy_scores_list length:  30
dummy_scores_list length:  33
dummy_scores_list length:  36
dummy_scores_list length:  39
dummy_scores_list length:  42
dummy_scores_list length:  45
dummy_scores_list length:  48
dummy_scores_list length:  51
dummy_scores_list length:  54
dummy_scores_list length:  57
dummy_scores_list length:  60
60
150000
Serialized calibration data to outputs/cam/calib_dataGetcam460.json
